In [5]:
import sqc
import numpy as np
from exercises import C2NOT

Nbits=9

# Initial state
op0=sqc.operator(Nbits).H(0).Rz(0,0.5)
s0=op0*sqc.state(Nbits)
print("Initial physical state")
print(s0)

Initial physical state
   0.707107             * |000000000>
 + (0.620545+0.339005j) * |000000001>


In [62]:
def Spread(a,b,c,op):
    return op.CNOT(a,b).CNOT(a,c)

def Correct(a,b,c,op):
    return C2NOT(b,c,a,op.CNOT(a,b).CNOT(a,c))

def Reset(bits,op):
    for a in bits:
        op=op.M(a,0).IF(1).X(a).ENDIF()
    return op

# Create logical state
op1=Spread(0,1,2,
        Spread(3,4,5,
            Spread(6,7,8,
               Spread(0,3,6,sqc.operator(Nbits)).H(0).H(3).H(6))))

# Correction circuit
opc=Correct(0,3,6,Correct(0,1,2,Correct(3,4,5,Correct(6,7,8,sqc.operator(Nbits)))).H(
    0).H(3).H(6))

# Reset ancilla bits circuit
opr=Reset(range(1,9),sqc.operator(Nbits))

# Print logical state
s1=op1*s0
print("Logical state")
print(s1)

Logical state
   (0.469396+0.119856j)  * |000000000>
 + (0.0306044-0.119856j) * |000000111>
 + (0.0306044-0.119856j) * |000111000>
 + (0.469396+0.119856j)  * |000111111>
 + (0.0306044-0.119856j) * |111000000>
 + (0.469396+0.119856j)  * |111000111>
 + (0.469396+0.119856j)  * |111111000>
 + (0.0306044-0.119856j) * |111111111>


In [63]:
# General one-qubit error
phierr=0.9

for errbit in range(9):
    print("Errbit", errbit)

    op2=sqc.operator(Nbits).Rz(errbit,phierr).X(errbit).H(errbit)
    s2=op2*s1
    print("After phase error")
    print(s2)

    s3=opc*s2
    print("After correction circuit")
    print(s3)
    
    s4=opr*s3
    print("After ancilla reset")
    print(s4)
    
    before=s4[1]/s4[0]
    after=s0[1]/s0[0]
    print(before/after)

Errbit 0
After phase error
   (0.331913+0.0847513j)   * |000000000>
 + (-0.331913-0.0847513j)  * |000000001>
 + (0.0798399-0.0357306j)  * |000000110>
 + (0.0798399-0.0357306j)  * |000000111>
 + (0.0216406-0.0847513j)  * |000111000>
 + (-0.0216406+0.0847513j) * |000111001>
 + (0.139932+0.312678j)    * |000111110>
 + (0.139932+0.312678j)    * |000111111>
 + (0.0216406-0.0847513j)  * |111000000>
 + (-0.0216406+0.0847513j) * |111000001>
 + (0.139932+0.312678j)    * |111000110>
 + (0.139932+0.312678j)    * |111000111>
 + (0.331913+0.0847513j)   * |111111000>
 + (-0.331913-0.0847513j)  * |111111001>
 + (0.0798399-0.0357306j)  * |111111110>
 + (0.0798399-0.0357306j)  * |111111111>
After correction circuit
   (0.405402+0.195832j)   * |000000000>
 + (0.261887+0.366219j)   * |000000001>
 + (-0.0945975+0.195832j) * |000000110>
 + (-0.176904+0.126506j)  * |000000111>
 + (0.0945975-0.195832j)  * |001001000>
 + (0.176904-0.126506j)   * |001001001>
 + (-0.405402-0.195832j)  * |001001110>
 + (-0.26188